In [118]:
import pandas as pd

In [124]:
from pymongo import MongoClient
import time
import api_keys
import requests
import gdax
import json

api = api_keys.api_keys()

client = MongoClient(api.mongo)
# specify the database and collection`
db = client.gdax.gdaxws
print('connected to mongo collection')

# and store in local variables
key = api.api_key
b64secret = api.secret_key
passphrase = api.passphrase

class GAC(gdax.AuthenticatedClient):
    def buy(self, **kwargs):
        kwargs["side"] = "buy"
        r = requests.post(self.url + '/orders',
                          data=json.dumps(kwargs),
                          auth=self.auth,
                          timeout=30)

        print(r)
        return r.json()

    def sell(self, **kwargs):
        kwargs["side"] = "sell"
        r = requests.post(self.url + '/orders',
                          data=json.dumps(kwargs),
                          auth=self.auth,
                          timeout=30)

        print('SOLD !!')
        print(r)
        db.orders.insert_one(r.json())
        return r.json()

    def getorders(self):
        """{'id': 'e22c9172-0276-47f7-b774-2559784c26aa', 'price': '999.85000000', 
            'size': '0.01000000', 'product_id': 'BTC-EUR', 'side': 'buy', 
            'type': 'limit', 'time_in_force': 'GTC', 'post_only': True,
            'created_at': '2018-08-17T14:57:18.551778Z', 'fill_fees': '0.0000000000000000',
            'filled_size': '0.00000000', 'executed_value': '0.0000000000000000', 
            'status': 'open', 'settled': False}"""
    
        self.openorders = client.get_orders()[0]
        self.df_openorders = pd.DataFrame(self.openorders)
        
    def getbalances(self):
        """{'id': '459d001f-0391-4e97-89e7-ae474275e2c9', 'currency': 'BTC',
            'balance': '0.0530287336346057', 'available': '0.0530287336346057',
            'hold': '0.0000000000000000', 'profile_id': '5100622b-3ed2-49e4-9810-c28fb96d30b3'} """

        self.balances = client.get_accounts()
        self.df_balances = pd.DataFrame(self.balances)

connected to mongo collection


In [125]:
client = GAC(key=key,b64secret=b64secret,
                 passphrase=passphrase,
                 api_url="https://api.pro.coinbase.com")
    
client.getbalances()

EUR balance at 164.2057972936818500
BTC balance at 0.0530287336346057


In [126]:
client.getorders()

In [137]:
client.df_balances.query('currency == "EUR"')['available'].astype('float') > 100

2    True
Name: available, dtype: bool